In [1]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
!pip install Adam


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install --upgrade pip

ERROR: To modify pip, please run the following command:
C:\Users\surpl\anaconda3\envs\tensor-env\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 2.1/2.1 MB 1.1 MB/s eta 0:00:00


In [3]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [4]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=128,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=128,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])

print(emotion_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         1

In [7]:
# Train the neural network/model
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 128,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 128)

Epoch 1/50
224/224 [==============================] - 1217s 5s/step - loss: 1.8219 - accuracy: 0.2452 - val_loss: 1.8183 - val_accuracy: 0.2471
Epoch 2/50
224/224 [==============================] - 703s 3s/step - loss: 1.7265 - accuracy: 0.3049 - val_loss: 1.6414 - val_accuracy: 0.3739
Epoch 3/50
224/224 [==============================] - 630s 3s/step - loss: 1.6141 - accuracy: 0.3740 - val_loss: 1.5616 - val_accuracy: 0.4043
Epoch 4/50
224/224 [==============================] - 635s 3s/step - loss: 1.5528 - accuracy: 0.4024 - val_loss: 1.5045 - val_accuracy: 0.4301
Epoch 5/50
224/224 [==============================] - 635s 3s/step - loss: 1.5034 - accuracy: 0.4231 - val_loss: 1.4562 - val_accuracy: 0.4453
Epoch 6/50
224/224 [==============================] - 640s 3s/step - loss: 1.4579 - accuracy: 0.4401 - val_loss: 1.4388 - val_accuracy: 0.4459
Epoch 7/50
224/224 [==============================] - 640s 3s/step - loss: 1.4199 - accuracy: 0.4559 - val_loss: 1.3917 - val_accuracy: 0.469

In [8]:
acc=emotion_model_info.history['accuracy']
val_acc=emotion_model_info.history['val_accuracy']
loss=emotion_model_info.history['loss']
val_loss=emotion_model_info.history['val_loss']

In [9]:
print(max(acc))
print(min(loss))

0.7815331816673279
0.6037284731864929


In [10]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')